In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'
sys.path.append(HOME + 'collaboration/')

In [3]:
import pandas as pd
import json
import numpy as np
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler

from math import log2
from utils import entropy

/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,
/usr/lib/python3/dist

In [4]:
# Page history

In [5]:
with open(HOME+'features/activity_weasel.json','rb') as f:
    features_pos = json.load(f)
with open(HOME+'negative_features/activity_weasel.json','rb') as f:
    features_neg = json.load(f)

In [6]:
len(features_pos)

1322

In [7]:
len(features_neg)

1187

In [8]:
article_features_neg = pd.DataFrame([d['article'] for d in features_neg])
article_features_neg['label'] = 0
article_features_pos = pd.DataFrame([d['article'] for d in features_pos])
article_features_pos['label'] = 1

In [9]:
article_data = pd.concat([article_features_neg, article_features_pos])
labels = article_data['label']
feat = article_data.drop('label',axis=1)

In [10]:
len(article_data)

2509

In [11]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
8,frac_recent_revisions,0.256553,0.256553
13,recent_edit_size,0.111633,0.111633
11,concentration_ratio,0.109498,0.109498
14,recent_response_time,0.076485,0.076485
12,contribution_frac_entropy,-0.073308,0.073308
1,time_to_respond,0.070563,0.070563
9,top_contributor_frac,0.058334,0.058334
0,edit_size,0.051861,0.051861
7,num_editors,-0.041554,0.041554
6,num_revisions,-0.041465,0.041465


In [12]:
# linear model

In [13]:
hypers = {'solver':'lbfgs','max_iter':10000,'C':10000}

In [14]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [15]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.69870463155322

In [16]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[221,  73],
       [161, 173]])

In [17]:
f1_score(y_pred=y_pred, y_true=y_test)

0.596551724137931

In [18]:
# user-article

In [19]:
def aggregate_features(article_users):
    if len(article_users) == 0:
        return {}
    else:
        article_users = pd.DataFrame(article_users).drop('event_user_id',axis=1)
        means = {'mean_'+key:val for key, val in article_users.mean().items()}
        std = {'std_'+key:val for key, val in article_users.std().items()}
        max_ = {'max_'+key:val for key, val in article_users.max().items()}
        ent = {'frac_page_edits_ent': entropy(article_users.frac_page_edits)}
        features = {**means, **std, **max_, **ent}

        return features

In [20]:
user_article_features_pos = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_pos])
user_article_features_neg = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_neg])

In [21]:
user_article_features_pos['label'] = 1
user_article_features_neg['label'] = 0

user_article_data = pd.concat([user_article_features_neg,user_article_features_pos])
labels = user_article_data['label']
feat = user_article_data.drop('label',axis=1)

In [22]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
5,mean_frac_page_edits,0.168689,0.168689
24,frac_page_edits_ent,-0.088316,0.088316
12,std_num_edits,-0.067213,0.067213
2,mean_time_to_respond,0.066158,0.066158
3,mean_time_responded_to,0.063776,0.063776
21,max_frac_page_edits,0.063415,0.063415
13,std_frac_page_edits,0.063340,0.063340
4,mean_num_edits,-0.057230,0.057230
10,std_time_to_respond,0.049629,0.049629
11,std_time_responded_to,0.045661,0.045661


In [23]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.5541461412151067

In [25]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[176, 104],
       [192, 156]])

In [26]:
f1_score(y_pred=y_pred, y_true=y_test)

0.5131578947368421

In [27]:
# talk vol

In [28]:
with open(HOME+'features/talk_weasel.json','rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_weasel.json','rb') as f:
    talk_neg = json.load(f)

In [29]:
talk_vol_neg = pd.DataFrame([d['talk_volume'] for d in talk_neg])
talk_vol_neg['label'] = 0
talk_vol_pos = pd.DataFrame([d['talk_volume'] for d in talk_pos])
talk_vol_pos['label'] = 1

In [30]:
talk_vol = pd.concat([talk_vol_neg,talk_vol_pos])
labels = talk_vol['label']
feat = talk_vol.drop('label',axis=1)

In [31]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
1,frac_recent_revisions,0.066025,0.066025
5,mean_response_time,0.058586,0.058586
3,top_contributor_frac,0.030387,0.030387
2,num_editors,-0.029847,0.029847
0,num_revisions,-0.021363,0.021363
6,page_talk_ratio,-0.019589,0.019589
4,mean_edit_size,-0.007681,0.007681


In [32]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [33]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.5344075193017791

In [34]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[104, 193],
       [100, 231]])

In [35]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6119205298013244

In [36]:
# talk lang

In [37]:
with open(HOME+'features/talk_weasel.json','rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_weasel.json','rb') as f:
    talk_neg = json.load(f)

In [38]:
def aggregate_language(talk_feat):
    if len(talk_feat) == 0:
        return {}
    else:
        all_utts = pd.DataFrame(list(np.concatenate(list(talk_feat.values()))))
        means = {'mean_'+key:val for key, val in all_utts.mean().items()}
        std = {'std_'+key:val for key, val in all_utts.std().items()}
        max_ = {'max_'+key:val for key, val in all_utts.max().items()}
        features = {**means, **std, **max_}

        return features

In [39]:
talk_lang_neg = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_neg])
talk_lang_neg['label'] = 0

In [40]:
len(talk_lang_neg)

1187

In [41]:
talk_lang_pos = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_pos])
talk_lang_pos['label'] = 1

In [42]:
len(talk_lang_pos)

1322

In [43]:
talk_lang = pd.concat([talk_lang_neg,talk_lang_pos])
labels = talk_lang['label']
feat = talk_lang.drop('label',axis=1)

In [44]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
11,mean_politeness_markers_==1st_person_start==,-0.030833,0.030833
58,max_politeness_markers_==Indirect_(greeting)==,-0.030296,0.030296
10,mean_politeness_markers_==1st_person==,-0.025049,0.025049
44,max_politeness_markers_==Please==,-0.023966,0.023966
45,max_politeness_markers_==Please_start==,-0.022454,0.022454
14,mean_politeness_markers_==Indirect_(greeting)==,-0.022230,0.022230
46,max_politeness_markers_==HASHEDGE==,-0.019711,0.019711
36,std_politeness_markers_==Indirect_(greeting)==,-0.019401,0.019401
55,max_politeness_markers_==1st_person_start==,-0.018589,0.018589
48,max_politeness_markers_==Hedges==,-0.016511,0.016511


In [45]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.42005538255550934

In [47]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 43, 268],
       [ 62, 255]])

In [48]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6071428571428572

In [49]:
# editors

In [50]:
def aggregate_editor(editor_feat):
    if len(editor_feat) == 0:
        return {}
    else:
        all_utts = pd.DataFrame(editor_feat).drop('event_user_id',axis=1)
        means = {'mean_'+key:val for key, val in all_utts.mean().items()}
        std = {'std_'+key:val for key, val in all_utts.std().items()}
        max_ = {'max_'+key:val for key, val in all_utts.max().items()}
        features = {**means, **std, **max_}

        return features

In [51]:
with open(HOME+'features/editorsweasel_dd.json','rb') as f:
    editors_pos = json.load(f)
    
with open(HOME+'negative_features/editorsweasel_v2.json','rb') as f:
    editors_neg = json.load(f)

In [52]:
labels = pd.read_csv('/srv/home/christinedk/wp_internship/data/negative_labels/weasel.csv')

In [53]:
editor_neg = pd.DataFrame([aggregate_editor(d['editor']) for d in editors_neg])
editor_neg['label'] = 0
editor_pos = pd.DataFrame([aggregate_editor(d['editor']) for d in editors_pos])
editor_pos['label'] = 1

In [54]:
len(editor_neg)

1187

In [55]:
len(editor_pos)

1322

In [56]:
editor_data = pd.concat([editor_neg,editor_pos])
labels = editor_data['label']
feat = editor_data.drop('label',axis=1)

In [57]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
29,max_contribution_frac_entropy,-0.097824,0.097824
20,max_num_groups,-0.073245,0.073245
9,mean_contribution_frac_entropy,-0.057362,0.057362
24,max_num_curr_blocks,-0.048621,0.048621
19,std_contribution_frac_entropy,0.045715,0.045715
23,max_num_past_blocks,-0.039697,0.039697
13,std_num_past_blocks,-0.037692,0.037692
28,max_talk_article_ratio,-0.033751,0.033751
27,max_time_since_registration,-0.033707,0.033707
26,max_num_reverts_of_others,-0.029298,0.029298


In [58]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [59]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.5345284278146248

In [60]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[150, 112],
       [186, 180]])

In [61]:
f1_score(y_pred=y_pred, y_true=y_test)

0.5471124620060791

In [62]:
# editor-editor

In [63]:
def aggregate(df):
    means = {'mean_'+key:val for key, val in df.mean().items()}
    std = {'std_'+key:val for key, val in df.std().items()}
    max_ = {'max_'+key:val for key, val in df.max().items()}
    features = {**means, **std, **max_}
    return features

def aggregate_collab(editor_feat):
    if len(editor_feat['undirected']) == 0:
        features = {}
        
    else:
        directed = pd.DataFrame(editor_feat['directed']).drop(['event_user_id','event_user_id_r'],axis=1)
        undirected = pd.DataFrame(editor_feat['undirected']).drop('pair',axis=1)

        dir_features = aggregate(directed)
        undir_features = aggregate(undirected)

        features = {**dir_features,**undir_features}

    return features

In [64]:
collab_neg = pd.DataFrame([aggregate_collab(d['collaboration']) for d in editors_neg])
collab_neg['label'] = 0
collab_pos = pd.DataFrame([aggregate_collab(d['collaboration']) for d in editors_pos])
collab_pos['label'] = 1

In [65]:
collab_data = pd.concat([editor_neg,editor_pos])
labels = collab_data['label']
feat = collab_data.drop('label',axis=1)

In [66]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
29,max_contribution_frac_entropy,-0.097824,0.097824
20,max_num_groups,-0.073245,0.073245
9,mean_contribution_frac_entropy,-0.057362,0.057362
24,max_num_curr_blocks,-0.048621,0.048621
19,std_contribution_frac_entropy,0.045715,0.045715
23,max_num_past_blocks,-0.039697,0.039697
13,std_num_past_blocks,-0.037692,0.037692
28,max_talk_article_ratio,-0.033751,0.033751
27,max_time_since_registration,-0.033707,0.033707
26,max_num_reverts_of_others,-0.029298,0.029298


In [67]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [68]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.5266559129967232

In [69]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[108, 201],
       [110, 209]])

In [70]:
f1_score(y_pred=y_pred, y_true=y_test)

0.5733882030178327

In [71]:
# combined

In [72]:
labels = article_data['label']

feat = pd.concat([ds.reset_index(drop=True).drop('label',axis=1) 
                  for ds in [article_data,talk_vol,talk_lang,user_article_data,editor_data,collab_data]],axis=1)

In [73]:
len(feat)

2509

In [74]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
13,recent_edit_size,0.072737,0.072737
7,num_editors,0.070929,0.070929
17,num_editors,0.066050,0.066050
12,contribution_frac_entropy,0.065525,0.065525
6,num_revisions,0.061432,0.061432
3,curr_size,0.060503,0.060503
5,last_edit_size,0.060503,0.060503
112,frac_page_edits_ent,0.055904,0.055904
15,num_revisions,0.050276,0.050276
97,std_next_edit_size,0.048378,0.048378


In [75]:
scaler = StandardScaler()
model = LogisticRegression(max_iter=500, fit_intercept=True,solver='saga')#,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [76]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

/usr/lib/python3/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.6358090185676393

In [77]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[196,  94],
       [160, 178]])

In [78]:
f1_score(y_pred=y_pred, y_true=y_test)

0.5836065573770491

In [79]:
# Thoughts:

# article:
# comparing to same pages means some features are less relevant; eg. num_editors, mean edit_size, top_contributor_frac, etc
# recency features should be more important 
#     - calculate all features for recent revisions only as well?


# user-article
# empty for some negatives? sample page': 189559,'date': '2003-02-28 16:09:01'
# revert features are incorrect, need to re-export

In [80]:
# Editor